# Document Conversion - Quick start

## Getting started

The [Deep Search Toolkit](https://ds4sd.github.io/deepsearch-toolkit/) allows document conversion with the following few lines of code. It's that simple! For more info or step-by-step guide:
- Visit https://ds4sd.github.io/deepsearch-toolkit/guide/convert_doc/
- Follow this example notebook

### Set notebook parameters

In [4]:
from dsnotebooks.settings import ProjectNotebookSettings

# notebook settings auto-loaded from .env / env vars
notebook_settings = ProjectNotebookSettings()

PROFILE_NAME = notebook_settings.profile  # the profile to use
PROJ_KEY = notebook_settings.proj_key  # the project to use

# default project_key = 1234567890abcdefghijklmnopqrstvwyz123456

### Import example dependencies

In [5]:
import deepsearch as ds
import json

from pathlib import Path
from zipfile import ZipFile

from deepsearch.documents.core.export import export_to_markdown
from IPython.display import display, Markdown, HTML, display_html

### Connect to Deep Search

In [6]:
api = ds.CpsApi.from_env(profile_name=PROFILE_NAME)

In [ ]:
output_dir = Path("./converted_docs")

documents = ds.convert_documents(
    api=api,
    proj_key=PROJ_KEY,
    source_path="../../data/samples/2206.01062.pdf",
    progress_bar=True,
)
documents.download_all(result_dir=output_dir)
info = documents.generate_report(result_dir=output_dir)
print(info)

In [ ]:
# Iterare output files and visualize the output
for output_file in output_dir.rglob("json*.zip"):
    with ZipFile(output_file) as archive:
        all_files = archive.namelist()
        for name in all_files:
            if not name.endswith(".json"):
                continue

            basename = name.rstrip(".json")
            doc_json = json.loads(archive.read(f"{basename}.json"))
            doc_md = export_to_markdown(doc_json)

            ofile = output_dir / f"{basename}.md"
            with ofile.open("w") as fw:
                fw.write(doc_md)

            print(f"writing {ofile}")

In [ ]:
# display last document
# display(Markdown(doc_md))

---

**There's more!**

The Deep Search Toolkit provides utility functions which can convert documents from different type of inputs.

- From a single url
- From a list of urls. In this case, the toolkit will launch a batch processing with all tasks.
- From a local PDF file
- From a local zip archive containing PDF files.
- From a local folder containing PDF files. In this case, the toolkit is packaging the files into batches and creates multiple zip archives.


---

## Let's explore document conversion

### Single URL

In [ ]:
documents = ds.convert_documents(
    api=api,
    proj_key=PROJ_KEY,
    urls="https://arxiv.org/pdf/2206.00785.pdf",
    progress_bar=True,
)

In [ ]:
# let's check what happened.
# we generate a csv report about the conversion task and store it locally
result_dir = "./converted_docs/"
info = documents.generate_report(result_dir=result_dir)
print(info)

The saved report may help in debugging and analysing the conversion task

In [ ]:
# let's download all the converted documents:
documents.download_all(result_dir=result_dir, progress_bar=True)

In [ ]:
# the documents object stores some additional info like:
documents.statuses, documents.task_ids

### Multiple URLs

In [ ]:
# let's create a list of urls we want to convert:
urls = ["https://arxiv.org/pdf/2206.00785.pdf", "https://arxiv.org/pdf/2206.01062.pdf"]

In [ ]:
# Process multiple urls
documents = ds.convert_documents(
    api=api, proj_key=PROJ_KEY, urls=urls, progress_bar=True
)

In [ ]:
# as before we can use the documents object to download all jsons. We can also iterate over them individually.
for doc in documents:
    doc.download(result_dir=result_dir, progress_bar=True)

### Process local file

In [ ]:
documents = ds.convert_documents(
    api=api,
    proj_key=PROJ_KEY,
    source_path="../../data/samples/2206.01062.pdf",
    progress_bar=True,
)

## Process folder of files

In [ ]:
documents = ds.convert_documents(
    api=api, proj_key=PROJ_KEY, source_path="../../data/samples", progress_bar=True
)

In [ ]:
info = documents.generate_report(result_dir)
print(info)

In [ ]:
# let's download all the converted documents:
documents.download_all(result_dir=result_dir, progress_bar=True)

---

## What's next?

Explore other examples which demonstrate possible use cases of the document conversion

- Visualize the text bounding boxes
- Extract figures
- Convert document to epub for your e-reader
